In [1]:
import json
from lib.generate import *
generator = Llama13BGenerator()
from time import sleep
from tqdm import tqdm
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download("punkt")
nltk.download('stopwords')
stops = stopwords.words("english")

import stanza
nlp = stanza.Pipeline(lang='en', processors='tokenize,ner')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/chaosarium/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/chaosarium/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/Users/chaosarium/anaconda3/envs/llama_hw/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-03-08 20:36:37 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2024-03-08 20:36:37 INFO: Downloaded file to /Users/chaosarium/stanza_resources/resources.json
2024-03-08 20:36:37 WARNING: Language en package default expects mwt, which has been added
2024-03-08 20:36:37 INFO: Loading thes

In [6]:
# file io is by copilot
def augment_test_set(name) -> None:
    count = 0
    with open(f'../annotations/{name}.json') as file:
        data = json.load(file)
        
    for i in tqdm(range(len(data))):
    # for i in range(len(data)):
        Q = data[i]["Q"]
        Q_aug = Q
        
        query_tok = [word.lower() for word in word_tokenize(Q) if word.lower() not in stops]
        ents = nlp(Q).ents
        do_augment = False
        if len(ents) < 1:
            do_augment = True
        
        for ent in ents:
            if ent.type in ['WORK_OF_ART', 'PRODUCT']:
                do_augment = False
        
        
        if do_augment:
            # print(f"augmenting: {Q}")
            # print(ents)
            Q_aug, Q_aug_prompt = generator.augment_query(Q)
            if 'Search Result:' in Q_aug:
                Q_aug = ''.join(Q_aug.split('Search Result:')[1:])
            Q_aug = Q_aug.strip()
            sleep(0.5)
            count += 1
            data[i]["Q_aug"] = Q + '\n' + Q_aug
            data[i]["Q_aug_prompt"] = Q_aug_prompt
        else:
            data[i]["Q_aug"] = Q
            data[i]["Q_aug_prompt"] = None

    print(f"augmented {count} for {name}")
    
    with open(f'../annotations/{name}_aug.json', 'w') as file:
        json.dump(data, file, indent=2)

In [7]:
augment_test_set('toyQAs-on-papers')
augment_test_set('leon-annotated')
augment_test_set('karina-annotated')
augment_test_set('papersQAs')
augment_test_set('coursesQAs')
augment_test_set('directoryQAs')

100%|██████████| 4/4 [00:03<00:00,  1.28it/s]


augmented 1 for toyQAs-on-papers


100%|██████████| 30/30 [00:22<00:00,  1.35it/s]


augmented 7 for leon-annotated


100%|██████████| 30/30 [00:04<00:00,  6.59it/s]


...

augmented 74 for papersQAs


100%|██████████| 300/300 [01:26<00:00,  3.46it/s]


augmented 22 for coursesQAs


100%|██████████| 300/300 [03:05<00:00,  1.62it/s]

augmented 56 for directoryQAs
